In [2]:
from pathlib import Path
import pandas as pd
import json
import re

def clean_string(s):
    # Remove control characters and clean up whitespace
    if pd.notna(s):
        return re.sub(r'[\x00-\x1F\x7F]', '', str(s)).strip()
    return ""

def format_array(value):
    if pd.isna(value):
        return []
    items = [clean_string(item) for item in str(value).replace("'", "").split('|')]
    return [item for item in items if item]

def safe_filename(name):
    return re.sub(r'[^\w\-.]', '_', name.lower().replace(' ', '_'))

def excel_to_json_files(df, output_dir='output'):
    Path(output_dir).mkdir(exist_ok=True)
    field_names = [name.strip() for name in df.iloc[:, 0] if pd.notna(name)]
    print(f"Field names: {field_names}")  # Debug

    for col_idx in range(1, len(df.columns)):
        try:
            data_dict = {}
            for field, value in zip(field_names, df.iloc[:, col_idx]):
                if pd.isna(value):
                    continue
                if "Hyperlinked Terms Associated" in field:
                    data_dict[field] = format_array(value)
                elif "Street addresses associated" in field:
                    data_dict[field] = format_array(value)
                else:
                    data_dict[field] = clean_string(value)

            if data_dict.get("Title"):
                title = clean_string(data_dict["Title"])
                file_name = f"{safe_filename(title)}_{col_idx}.json"
                file_path = Path(output_dir) / file_name
                try:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(data_dict, f, indent=4, ensure_ascii=False)
                    print(f"Created {file_name}")
                except json.JSONDecodeError as e:
                    print(f"Invalid JSON for {title}: {e}")
            else:
                print(f"Column {col_idx}: Missing 'Title', skipping...")
        except Exception as e:
            print(f"Error in column {col_idx}: {e}")

if __name__ == "__main__":
    excel_file_path = "Book15.xlsx"
    output_directory = "VV"

    try:
        print("Reading Excel file...")
        df = pd.read_excel(excel_file_path)
        print(f"Successfully read Excel file with {len(df)} rows and {len(df.columns)} columns")
        excel_to_json_files(df, output_dir=output_directory)
    except FileNotFoundError:
        print(f"File not found: {excel_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


Reading Excel file...
Successfully read Excel file with 15 rows and 25 columns
Field names: ['Title', 'Descriptive Text', 'Creator (Photographer)', 'Source', 'Bibliography', 'Author of Text', 'Hyperlinked Terms Associated', 'Historical Sources Associated (Title, Source, and Link)', 'Location markers (lat/long)', 'neighborhood names associated', 'Street addresses associated', 'time periods (YYYY-YYYY)', 'Keywords associated with the Chicago Collections Consortium', 'Table?']
Created vacation_spots_1.json
Created valparaiso_university_2.json
Created valparaiso__in_3.json
Created vaudeville_4.json
Created veluchamy_enterprises_5.json
Created venezuelans_6.json
Created vernon_hills__il_7.json
Created veterans__hospitals_8.json
Created vice_commissions_9.json
Created vice_districts_10.json
Created victims_of_the_january_1909_68th_street_water_intake_crib_disaster_11.json
Created victor_adding_machine_co._12.json
Created vietnamese_13.json
Created view_east_toward_buckingham_fountain_and_lak

In [6]:
for col_idx in range(1, len(df.columns)):
    try:
        data_dict = {}
        for field, value in zip(field_names, df.iloc[:, col_idx]):
            if pd.isna(value):
                continue
            if "Hyperlinked Terms Associated" in field:
                data_dict[field] = format_array(value)
            else:
                data_dict[field] = clean_string(value)

        # Construct file name
        title = clean_string(df.iloc[0, col_idx])  # First row as title
        file_name = Path(output_dir) / f"{title.replace(' ', '_').lower()}.json"

        # Debug log: processing file
        print(f"Processing file: {file_name}...")

        # Always write/overwrite the file
        with open(file_name, 'w') as json_file:
            json.dump(data_dict, json_file, indent=4)

    except Exception as e:
        print(f"Error processing column {col_idx}: {e}")


Error processing column 1: name 'field_names' is not defined
Error processing column 2: name 'field_names' is not defined
